# 7.1 투표 기반 분류기 

모든 분류기가 완벽하게 독립적이고 오차에 상관관계가 없을 경우 큰 수의 법칙에 의해 voting의 결과가 더 좋아진다!
(극단적으로 51% 정확도를 가진 1000개의 분류기로 앙상블 모델을 구축한다면 75%의 정확도를 기대할 수 있습니다)

다만 이런 가정은 현실에서는 같은 데이터로 훈련시키기 떄문에 이러한 가정이 맞지 않습니다.
분류기들이 같은 종류의 오차를 만들기 떄문에 잘못된 클래스가 다수일 확률이 높아집니다.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [2]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')

In [3]:
voting_clf.fit=(X_train,y_train)
voting_clf

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                             

In [4]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.88
SVC 0.896


TypeError: 'tuple' object is not callable

In [ ]:
voting_clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(voting_clf.__class__.__name__, accuracy_score(y_test, y_pred))

# 7.2 배깅과 페이스팅
vagging (bootstrap aggregating): 중복허용해서 리샘플링 (배깅만 한 예측기를 위해 같은 훈련샘플을 여러 번 샘플링 가능)

pasting : 중복을 허용하지 않고 샘플링    
    
    
개별 예측기는 원본 훈련세트로 훈련시킨 것보다 크게 편향되어 있지만 수집함수를 통과하면 편향과 분산이 모두 감소,

일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬때와 비교해 편향은 비슷하지만 분산은 줄어듭니다.    

## 7.2.1 사이킷런의 배깅과 페이스팅

In [5]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf= BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples =100, bootstrap=True, n_jobs=-1)# n_jobs는 학스에 사용될 코어수를 지정(-1로 지정하면 모든 코어수 사용)
bag_clf.fit(X_train,y_train)
y_pred =bag_clf.predict(X_test)

bootstraping 은 각 예측기가 학습하는 서브셋의 다양성을 증가시키므로 배깅이 페이스팅보다 편향이 조금 더 높다. 

하지만 다양성을 추가하는 것은 예측기들의 상관관계를 줄이므로 앙상블의 분산을 감소시킨다.

# 7.2.2 obb 평가

배깅을 사용할때 중복을 허용하므로 선택되지 않은 훈련샘플이 있습니다 

이를 oob 샘플 (out of bag)이라고 합니다. 예측기마다 oob는 모두 다릅니다

따라서 이를 검증셋으로 사용할 수 있고 앙상블의 평가는 각 예측기의 oob의 평가를 평균하여 얻습니다.

BaggingClassifier에서 oob_score=True로 설정하면 훈련이 끝난후 자동으로 obb평가를 수행합니다.

In [11]:
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(),n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True
)

In [12]:
bag_clf.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=None,


In [13]:
bag_clf.oob_score_ # oob에서의 평가

0.9013333333333333

In [14]:
from sklearn.metrics import accuracy_score

y_pred=bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.888

In [15]:
bag_clf.oob_decision_function_

array([[0.3919598 , 0.6080402 ],
       [0.3038674 , 0.6961326 ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.08235294, 0.91764706],
       [0.44262295, 0.55737705],
       [0.03553299, 0.96446701],
       [0.98958333, 0.01041667],
       [0.97536946, 0.02463054],
       [0.7431694 , 0.2568306 ],
       [0.01265823, 0.98734177],
       [0.79444444, 0.20555556],
       [0.82673267, 0.17326733],
       [0.9558011 , 0.0441989 ],
       [0.05780347, 0.94219653],
       [0.        , 1.        ],
       [0.98369565, 0.01630435],
       [0.95721925, 0.04278075],
       [0.98837209, 0.01162791],
       [0.01587302, 0.98412698],
       [0.31360947, 0.68639053],
       [0.90322581, 0.09677419],
       [1.        , 0.        ],
       [0.9895288 , 0.0104712 ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.63684211, 0.36315789],
       [0.

# 7.3 랜덤패치와 서브스페이스

훈련 특성과 샘플을 모두 샘플링 하는 것을 랜덤 패치 방식(random patches method) 이라고 한다. (이미지와 같은 고차원의 데이터셋을 다루는데 유용)

훈련 샘플을 모두 사용하고 (bootstrap=False, max_samples=1.0) 특성은 샘플링(bootsrap_features=True, max_features<1.0)하는 것을
랜덤 서브스페이스 방식(random subspace method)이라고 한다

-> 특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춰줍니다.

# 7.4 랜덤 포레스트

In [17]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf=rnd_clf.predict(X_test)

bagging classifier는 결정트리 의외의 모델에 적용할때 사용가능!

랜덤 포레스트 알고리즘은 전체 특성 중 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입합니다.

이는 결국 트리를 더욱 다양하게 만들고 편향을 손해보는 대신 분산을 낮추어 전체적으로 더 휼륭한 모델을 만들어냅니다.

In [20]:
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

# 7.4.1 엑스트라 트리 

랜덤 포레스트에서 트리를 만들 때 각 노드는 무작위로 특성의 서브셋을 만들어 분할에 사용합니다. 
트리를 더욱 무작위하게 만들기 위해 최적의 임계값을 찾는 대신 후보특성을 사용해 무작위로 분할한 다음 그 중에서 최상의 분할을 선택합니다.
-> 익스트라 랜덤 트리 앙상블 (extremly ranodomized trees) 또는 extra trees 라고 한다. 

(편향이 늘어나지만 분산이 줄어든다. 모든 노드에서 최적의 임계값을 찾을 필요가 없으므로 랜덤 포레스트보다 엑스트라 트리가 훨씬 빠르다.)

랜포와 엑스트라 트리 둘다 시도해서 비교해보고 그리드 서치로 하이퍼 파라미터 튜닝함

ExtraClassifier

# 7.4.2 특성 중요도 

사이킷런은 어떤 특성을 사용한 노드가 모든 트리에 걸쳐서 평균적으로 불순도를 얼마나 감소시켰는지를 기준으로 중요도를 측정

-> 가중치 평균이며 각 노드의 가중치는 연관된 훈련샘플의 수와 같음

학습이 끝난후 중요도가 전체 합이 1이 되도록 결과값을 정규화함 

결정 트리의 중요도(전체합이 1이 되도록 정규화) = 현재 노드의 샘플 비율 x 불순도 - 왼쪽노드의 샘플 비율 x 불순도 - 오른쪽노드의 샘플 비율 x 불순도

랜덤포레스트의 중요도 -> 각 결정트리의 특성 중요도를 모두 계산하여 더한 후 트리 개수로 나눔

In [24]:
from sklearn.datasets import load_iris

iris=load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500,n_jobs=-1)
rnd_clf.fit(iris["data"],iris["target"])

for name,score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name,score)    

sepal length (cm) 0.09792045332721849
sepal width (cm) 0.02286977670844244
petal length (cm) 0.40378981429486427
petal width (cm) 0.47541995566947487


# 7.5 부스팅

약한 학습기를 여러개 연결하여 강한 학습기를 만드는 앙상블 방법

(보통 잔차를 업데이트하여 다음 데이터셋을 구성)

## 7.5.1 에이다부스트

알고리즘이 기반이 되는 첫번째 분류기를 훈련세트에 적합시키고 예측치를 만듬

잘못 분류된 훈련샘플의 가중치를 상대적으로 높임,

두번째 분류기는 업데이트된 가중치를 사용해 훈련세트에서 훈련하고 다시 예측을 만듬 

위 방법을 계속 진행

caution!! 연속된 학습 기법은 각 예측기가 이전 예측기의 훈련후 학습될 수 있기 때문에 병렬화(or 분할)을 할 수 없습니다. 
따라서 배깅이나 페이스팅만큼 확장성이 높지 않습니다.

In [27]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf=AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5
)
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=1,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                          

In [28]:
y_pred=ada_clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.896

## 7.5.2 그래디언트 부스팅

이전 예측기가 만든 잔여 오차(residual error)에 새로운 예측기를 학습시킵니다.


In [42]:
import numpy as np
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [43]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 =DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=2,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [44]:
y2=y-tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=2,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [45]:
y3=y2-tree_reg1.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=2,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [46]:
import numpy as np
X_new = np.array([[0.8]])

In [47]:
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [48]:
y_pred

array([0.03991297])

In [50]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=1.0, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

learning_rate는 매개변수가 각 트리의 기여정도를 조정한다
이를 낮게 설정하면(많은 트리가 필요)일반적으로 예측 성능이 좋아집니다.

->축소(shrinkage)라는 규제방법


In [55]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X,y)

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train,y_train)

errors = [mean_squared_error(y_val,y_pred)
         for y_pred in gbrt.staged_predict(X_val)] # stagged_predict -> 훈련의 각 단계에서 아아상블에 의해 만들어진 예특기를 순회하는 반복자를 반환
bst_n_estimators = np.argmin(errors)+1 # 최적 트리수!!

gbrt_best=GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators )
gbrt_best.fit(X_train, y_train)
# warm_start =True 로 설정하면 fit() 메서드가 호출될떄 기존 트리를 유지하고 훈련을 추가할 수 있도록 한다.

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=83,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [58]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break  # early stopping

GradientBoostingRegressor는 각 트리가 훈련될때 사용할 훈련 샘플의 비율을 지정할 수 있습니다

(sub_sample = 0.25라고 하면 각 트리는 무작위로 선택된 25%의 훈련샘플로 학습됨)

-> 이라한 기법을 확률적 그레디언트 부스팅 기법이라 함


In [60]:
import xgboost

xgb_reg=xgboost.XGBRegressor()
xgb_reg.fit(X_train,y_train)
y_pred = xgb_reg.predict(X_val)


[20:06:34] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [61]:
xgb_reg.fit(X_train, y_train,
                eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

[20:07:53] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.308403
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.280384
[2]	validation_0-rmse:0.255261
[3]	validation_0-rmse:0.232697
[4]	validation_0-rmse:0.211968
[5]	validation_0-rmse:0.192675
[6]	validation_0-rmse:0.177464
[7]	validation_0-rmse:0.163445
[8]	validation_0-rmse:0.151159
[9]	validation_0-rmse:0.140248
[10]	validation_0-rmse:0.130624
[11]	validation_0-rmse:0.121576
[12]	validation_0-rmse:0.113691
[13]	validation_0-rmse:0.106864
[14]	validation_0-rmse:0.101159
[15]	validation_0-rmse:0.095552
[16]	validation_0-rmse:0.091277
[17]	validation_0-rmse:0.087887
[18]	validation_0-rmse:0.084786
[19]	validation_0-rmse:0.081612
[20]	validation_0-rmse:0.079539
[21]	validation_0-rmse:0.077531
[22]	validation_0-rmse:0.076274
[23]	validation_0-rmse:0.074814
[24]	validatio